<a href="https://colab.research.google.com/github/fg0611/data-professionalization/blob/main/data-science/population_versus_ghg_emmissions_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GHG and World Human Population datasets analysis and ml prediction model 

In [170]:
# population dataset contains year, country, number
# ghg dataset contains year, Entity (this one contains countries, continents and regions) has to be cleaned up, also GHG emissions CO2eq in tons  

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
cd /content/drive/MyDrive/00 DEV CODING/datasets

/content/drive/MyDrive/00 DEV CODING/datasets


In [213]:
import pandas as pd

In [214]:
ghg = pd.read_csv('total-ghg-emissions.csv')

## checking for nulls on every column
null_columns=ghg.columns[ghg.isnull().any()]
ghg[null_columns].isnull().sum()
## Code has 330 nulls

## checking for nulls on one column
## print(ghg[ghg["Code"].isnull()][null_columns])

## checking for nulls on all columns
## print(ghg[ghg.isnull().any(axis=1)][null_columns].head())

Code    330
dtype: int64

In [215]:
## ghg = pd.read_csv('total-ghg-emissions.csv')
ghg.dropna(inplace=True)
ghg = ghg[(ghg['Code'] != "OWID_WRL") & (ghg['Year'] >= 1990)]
ghg.rename(columns={"Entity": "country", "Year": "year", "Total including LUCF": "ghg"}, inplace=True)
ghg.drop(columns=["country", "Code"], inplace=True)
ghg = ghg.groupby(["year"], as_index=False).sum()
ghg.head()

,year,ghg
0,1990,3.133164e+10
1,1991,3.149132e+10
2,1992,3.149124e+10
3,1993,3.164125e+10
4,1994,3.190472e+10


In [216]:
null_columns=ghg.columns[ghg.isnull().any()]
ghg[null_columns].isnull().sum()

Series([], dtype: float64)

In [217]:
pop = pd.read_excel('population.xlsx').sort_values("year")
indices=range(len(pop))
pop.index=indices
pop = pop[pop['year'] >= 1990]
indices=range(len(pop))
pop.index=indices
# filter = pop["year"]>1990
# pop.where(filter, inplace = True)
pop.head()

,year,world population,density p/km2
0,1990,"5,327,231,061",36
1,1991,"5,414,289,444",36
2,1992,"5,498,919,809",37
3,1993,"5,581,597,546",37
4,1994,"5,663,150,427",38


In [218]:
pop_ghg = pop.merge(ghg, left_on='year', right_on='year')
# convert string to an integer
pop_ghg['world population'] = pop_ghg['world population'].str.replace('[^0-9]', '', regex=True).astype('int64')
pop_ghg.head()

,year,world population,density p/km2,ghg
0,1990,5327231061,36,3.133164e+10
1,1991,5414289444,36,3.149132e+10
2,1992,5498919809,37,3.149124e+10
3,1993,5581597546,37,3.164125e+10
4,1994,5663150427,38,3.190472e+10


In [224]:
from sklearn.linear_model import LinearRegression

# split data into features and target
X = pop_ghg[["year", "world population"]]
y = pop_ghg["ghg"]

# create and train the model
model = LinearRegression()
model.fit(X, y)

# make predictions for year 2100 with a pop of 10,900,000,000 or 10.9 billion
predictions2030 = model.predict([[2030, 8500000000]])
predictions2050 = model.predict([[2050, 9800000000]])

# print predictions
print(predictions2030)
# for 2030, ghg emission at 54billion tons CO2eq
print(predictions2050)
# for 2050, ghg emission at 65billion tons CO2eq

[5.43513759e+10]
[6.52321081e+10]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
